In [1]:
import pandas as pd

In [2]:
#Importing the dataset
data = pd.read_csv('Crop_recommendation.csv')

In [3]:
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [4]:
#Checking for missing values
data.isnull().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [5]:
#Checking for number of unique values in the target column
data['label'].value_counts()

label
rice           100
maize          100
chickpea       100
kidneybeans    100
pigeonpeas     100
mothbeans      100
mungbean       100
blackgram      100
lentil         100
pomegranate    100
banana         100
mango          100
grapes         100
watermelon     100
muskmelon      100
apple          100
orange         100
papaya         100
coconut        100
cotton         100
jute           100
coffee         100
Name: count, dtype: int64

In [6]:
x = data.iloc[:, :-1]


In [7]:
#Normalizing the features using StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)
x= pd.DataFrame(x)

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# Separate features and target
y = data['label']

# Convert target to one-hot encoding
y_one_hot = pd.get_dummies(y).values

In [9]:
# Storing the scaler model in a joblib file for future use
import joblib
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']

In [10]:
# Storing the label encoder in a joblib file
import joblib
joblib.dump(label_encoder, 'label_encoder.joblib')

['label_encoder.joblib']

In [11]:
# Train-test split with stratification
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x, y_one_hot, test_size=0.2, stratify=y_one_hot, random_state=42
)

In [12]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
#Creating the model using Keras
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

d:\projects\Crop Prediction\crop\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.1636 - loss: 3.0291 - val_accuracy: 0.4233 - val_loss: 2.6768
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4614 - loss: 2.5790 - val_accuracy: 0.6165 - val_loss: 2.0437
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6364 - loss: 1.9227 - val_accuracy: 0.7244 - val_loss: 1.3500
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7489 - loss: 1.2836 - val_accuracy: 0.8097 - val_loss: 0.8957
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8314 - loss: 0.8773 - val_accuracy: 0.8551 - val_loss: 0.6385
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8790 - loss: 0.6222 - val_accuracy: 0.9062 - val_loss: 0.4886
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9275 - loss: 0.4499 - val_accuracy: 0.9091 - val_loss: 0.3885
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9306 - loss: 0.3765 - val_accuracy: 0.9318 - val_loss

In [15]:
# Dumping the model in a joblib file
import joblib
joblib.dump(model, 'model.joblib')

['model.joblib']

In [16]:
model= joblib.load(open('model.joblib', 'rb'))

In [17]:
# Evaluate the model using a random sample
import numpy as np
sample_input = np.array([[90, 42, 43, 20.88, 82.00, 6.5, 203]])  # Example input
sample_input_scaled = scaler.transform(sample_input)
prediction = model.predict(sample_input_scaled)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])

d:\projects\Crop Prediction\crop\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step


In [18]:
np.argmax(prediction)

np.int64(20)

In [19]:
print(f"Recommended Crop: {predicted_label[0]}")

Recommended Crop: rice


In [20]:
# Evaluate the model using the test set
from sklearn.metrics import classification_report
import numpy as np
# Get predictions for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_test_classes = np.argmax(y_test, axis=1)  # Convert one-hot encoding to class labels

# Generate classification report
report = classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_)
print("Classification Report:\n", report)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  
Classification Report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        20
      banana       1.00      1.00      1.00        20
   blackgram       1.00      1.00      1.00        20
    chickpea       1.00      1.00      1.00        20
     coconut       1.00      1.00      1.00        20
      coffee       1.00      1.00      1.00        20
      cotton       1.00      1.00      1.00        20
      grapes       1.00      1.00      1.00        20
        jute       0.94      0.80      0.86        20
 kidneybeans       1.00      1.00      1.00        20
      lentil       1.00      1.00      1.00        20
       maize       1.00      1.00      1.00        20
       mango       1.00      1.00      1.00        20
   mothbeans       1.00      1.00      1.00        20
    mungbean       1.00      1.00      1.00        20
   muskmelon       1.00      0.95      0.97        20
      orange    

In [21]:
label_encoder.classes_[0]

'apple'